In [1]:
import numpy as np
import pickle

# Matricies

This notebook uses many images from the excellent [A Visual Intro to NumPy and Data Representation](https://jalammar.github.io/visual-numpy/) from [Jay Alammar](https://jalammar.github.io/).

In the first notebook ([vector.ipynb](https://github.com/ADGEfficiency/teaching-monolith/blob/master/numpy/1.vector.ipynb)) we dealt with vectors (one dimensional). 

Now we deal with **Matricies** - arrays with two dimensions.

$\textbf{A}_{2, 2} = \begin{bmatrix}A_{1, 1} & A_{1, 2} \\ A_{2, 1} & A_{2, 2}\end{bmatrix}$

- two dimensional
- uppercase, bold $\textbf{A}_{m, n}$
- $A_{1, 1}$ = first element
- area
- tabular data

## Reshaping

Now that we have multiple dimensions, we need to start considering shape.

We can see the shape using `.shape`

In [2]:
data = np.arange(16)

data.shape

(16,)

And the number of elements using `.size`

In [3]:
data.size

16

The **shape** of a matrix becomes more than just an indication of the length.  We can change the shape using reshape:

In [4]:
data.reshape(4, 4)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

A very useful tool when reshaping is using `-1` - this is a free dimension that will be set to match the size of the data
- this is often set to the batch / number of samples dimension

In [5]:
data.reshape(2, -1)

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15]])

In [6]:
data.reshape(-1, 4)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

We can use `.reshape` to flatten

In [7]:
data.reshape(-1)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

We can also use `.flatten`

In [8]:
data.flatten()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

And finally `.ravel`

In [9]:
data.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

`.flatten` always returns a copy - `.ravel` doesn't (if it can)

Closely related to a reshape is the **transpose**, which flips the array along the diagonal:

<img src="assets/trans.png" alt="" width="300"/>

In [10]:
np.arange(0, 6).reshape(3, -1)

array([[0, 1],
       [2, 3],
       [4, 5]])

In [11]:
np.arange(0, 6).reshape(3, -1).T

array([[0, 2, 4],
       [1, 3, 5]])

Reshape is (usually) computationally **cheap** - to understand why we need to know a little about how a `np.array` is laid out in memory

## The `np.array` in memory

- the data is stored in a single block
- the shape is stored as a tuple

Why is storing in a single block (known as a contiguous layout) a good thing?
- to access the next value an the array 
- we just move to the next memory address
- length = defined by the data type

> ... storing data in a contiguous block of memory ensures that the architecture of modern CPUs is used optimally, in terms of memory access patterns, CPU cache, and vectorized instructions - [iPython coobook](https://ipython-books.github.io/45-understanding-the-internals-of-numpy-to-avoid-unnecessary-array-copying/)

Changing the shape only means changing the tuple 
- the layout of the data in memory is not changed

The operations that will change the memory layout are ones that change the order of the data - for example a transpose:

In [12]:
data = np.arange(10000000).reshape(5, -1)
res = %timeit -qo data.reshape((1, -1))
'{:.8f} seconds'.format(res.average)

'0.00000037 seconds'

In [13]:
data = np.arange(10000000).reshape(5, -1)
res = %timeit -qo data.T.reshape((1, -1))
'{:.8f} seconds'.format(res.average)

'0.02354572 seconds'

## Two dimensional indexing

<img src="assets/idx2.png" alt="" width="500"/>

In [14]:
data = np.random.rand(2, 3)
data

array([[0.28537847, 0.56546508, 0.74278854],
       [0.08762476, 0.41952309, 0.14012065]])

We specify both dimensions using a familiar `[]` syntax

`:` = entire dimension

In [15]:
#  first row
data[0, :]

array([0.28537847, 0.56546508, 0.74278854])

`-1` = last element

In [16]:
#  last column
data[:, -1]

array([0.74278854, 0.14012065])

### Two dimension aggregation

<img src="assets/agg-2d.png" alt="" width="900"/>

Now that we are working in two dimensions, we have more flexibility in how we aggregate
- we can specify the axis (i.e. the dimension) along which we aggregate

In [17]:
data

array([[0.28537847, 0.56546508, 0.74278854],
       [0.08762476, 0.41952309, 0.14012065]])

In [18]:
#  average over all the data
np.mean(data)

0.3734834320328672

In [19]:
#  average over the rows - so we end up with an array with one element per column (3)
np.mean(data, axis=0)

array([0.18650161, 0.49249409, 0.44145459])

In [20]:
#  average over the columns - so we end up with an array with one element per row (2)
np.mean(data, axis=1)

array([0.5312107 , 0.21575617])

By default `numpy` will remove the dimension you are aggregating over:

In [21]:
data

array([[0.28537847, 0.56546508, 0.74278854],
       [0.08762476, 0.41952309, 0.14012065]])

In [22]:
np.mean(data, axis=1).shape

(2,)

You can choose to keep this dimension using a `keepdims` argument:

In [23]:
np.mean(data, axis=1, keepdims=True).shape

(2, 1)

## Practical

Aggregate by variance `np.var` 
- over the rows
- over the columns
- over all data

## Two dimensional broadcasting

The general rule with broadcasting - dimensions are compatible when
- they are equal
- or when one of them is 1

<img src="assets/broad-2d.png" alt="" width="500"/>

In [24]:
data = np.arange(1, 7).reshape(3, 2)
data

array([[1, 2],
       [3, 4],
       [5, 6]])

In [25]:
data + np.array([0, 1, 1]).reshape(3, 1)

array([[1, 2],
       [4, 5],
       [6, 7]])

In [26]:
data + 1

array([[2, 3],
       [4, 5],
       [6, 7]])

## Matrix arithmetic

Can make arrays from nested lists:

In [27]:
np.array([[1, 2], [3, 4]])

array([[1, 2],
       [3, 4]])

We can add matricies of the same shape as expected:

<img src="assets/add-matrix.png" alt="" width="300"/>

In [28]:
#  more on ones_like below!
data + np.ones_like(data)

array([[2, 3],
       [4, 5],
       [6, 7]])

## Matrix multiplication

This kind of matrix multiplication will often **change the shape** of the array
- this is what happens in neural networks

<img src="assets/dot1.png" alt="" width="900"/>

This operation can be visualized:

<img src="assets/dot2.png" alt="" width="900"/>

In [29]:
data = np.array([1, 2, 3])

powers_of_ten = np.array([10**n for n in range(6)]).reshape(3, 2)

powers_of_ten

array([[     1,     10],
       [   100,   1000],
       [ 10000, 100000]])

This is done in numpy using either `np.dot()`:

In [30]:
np.dot(data, powers_of_ten)

array([ 30201, 302010])

Or calling the `.dot()` method on the array itself:

In [31]:
data.dot(powers_of_ten)

array([ 30201, 302010])

## Making arrays from nested lists

In [32]:
data = np.array([[1, 2], [3, 4]])
print(data)

[[1 2]
 [3 4]]


## Making arrays from shape tuples

The argument to these functions is a tuple

### `zeros`, `ones`, `full`

In [33]:
#  array of all zeros
np.zeros((2, 4))

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [34]:
#  array of all ones
np.ones((2, 4))

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [35]:
#  array filled with a value we choose
np.full((2, 4), 3)

array([[3, 3, 3, 3],
       [3, 3, 3, 3]])

### `zeros_like`, `ones_like`, `full_like`

Similar to counterparts above, except their shape is defined by another array:

In [36]:
parent = np.arange(10).reshape(2, 5)
parent

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [37]:
np.zeros_like(parent)

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [38]:
np.ones_like(parent)

array([[1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]])

In [39]:
np.full_like(parent, 3)

array([[3, 3, 3, 3, 3],
       [3, 3, 3, 3, 3]])

### `empty`

Similar to `zeros`, except the array is filled with garbage from RAM 
- this is a bit quicker than `zeros`

In [40]:
d = np.empty(4)
for e in range(4):
    d[e] = e
    
d

array([0., 1., 2., 3.])

### `eye`

Identity matrix :

In [41]:
np.eye(2)

array([[1., 0.],
       [0., 1.]])

The linear algebra verision of a 1

In [42]:
d = np.arange(4).reshape(2, 2)

d

array([[0, 1],
       [2, 3]])

In [43]:
np.dot(np.eye(2), d)

array([[0., 1.],
       [2., 3.]])

# Regression factors
The formula for the regression coefficients is

$\beta = (X'X)^{(-1)}X'Y $

But the data is a bit messed up, meaning that the format of the independent variables are saved in a flat array. That means we have a 1xN vector. I.e. the data was changed from that: 

<img src="assets/data_before.png" alt="" width="500"/>

to that:

<img src="assets/data_after.png" alt="" width="700"/>

The array contains the following variables: 

- Sale (in Dollars) - Amount of money received by the store
- Pack Size - Number of bottles per item
- State Bottle Cost - Cost of producing the bottle 
- Packs Sold - Amount of bottles sold
- Bottle Volume (in ml) - How many ml each bottle has



Question: Determine the regression coefficents of the following OLS regression

$Sale = \beta_0 + \beta_1 * (Pack Size) + \beta_2 * (State Bottle Cost) + \beta_3 * (Packs Sold) + \beta_4 * (Bottle Volume) + \epsilon $

## Answers
You are encouraged to use this upcoming line only for checking purposes

In [44]:
from Answers import beta_coefficients, stats_package
beta_coefficients()

array([-3.88928013e+01, -4.62402519e+00,  9.48100848e+00,  1.53183949e+01,
       -1.88215965e-02])

## Cross checking
Here we are cross checking with a statistics package

In [45]:
stats_package()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.779
Method:                 Least Squares   F-statistic:                 8.835e+04
Date:                Wed, 15 Jan 2020   Prob (F-statistic):               0.00
Time:                        20:04:45   Log-Likelihood:            -6.6361e+05
No. Observations:              100000   AIC:                         1.327e+06
Df Residuals:                   99995   BIC:                         1.327e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -38.8928      2.675    -14.537      0.000     -44.137     -33.649
x1            -4.6240      0.104    -44.531      0.000      -4.828      -4.421
x2             9.4810      0.092    102.755      0.000       9.300       9.662
x3            15.3184      0.026    586.137      0.000      15.267      15.370
x4            -0.0188      0.002    -12.034      0.000      -0.022      -0.016
==============================================================================
Omnibus:                   190007.483   Durbin-Watson:                   2.005
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      11161884404.416
Skew:                          13.336   Prob(JB):                         0.00
Kurtosis:                    1639.503   Cond. No.                     4.81e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.81e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""